<a href="https://colab.research.google.com/github/chimaobi-okite/Tourism_classification/blob/main/tourist_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Lets Build A Base Xgbosst model without any feature engineering and parameter tuning,.

Note: xgboost handles missing values for us.


Import the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import xgboost as xgb

In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/chimaobi-okite/Tourism_classification/main/Train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/chimaobi-okite/Tourism_classification/main/Test.csv')
var_def = pd.read_csv('https://raw.githubusercontent.com/chimaobi-okite/Tourism_classification/main/VariableDefinitions.csv')
ss = pd.read_csv('https://raw.githubusercontent.com/chimaobi-okite/Tourism_classification/main/SampleSubmission.csv')

In [3]:
## divide the dataset into features and target
X, y = train.drop(['cost_category','Tour_ID', 'country'], axis =1), train['cost_category']
## get onehot encoding of catgorical variable
X = pd.get_dummies(X)
X = X.rename(columns= {'age_group_65+':'age_group_65_plus','age_group_<18':'age_group_18_less'})
## get label categories
targets = {'Lower Cost':1, 'Low Cost':2, 'Normal Cost':3, 'High Cost': 4,'Higher Cost': 5, 'Highest Cost' : 6}
y = y.map(targets)
## split data to train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
## function to validate data
def validate( estimator, X, y,cv = 5):
  scores = cross_val_score(estimator= estimator, X =X, y = y, cv = cv, scoring = 'neg_log_loss')
  return scores.mean(), scores.std()

In [5]:
## validate xgbclassifier
model = xgb.XGBClassifier(seed = 42)
validate(model, X_train, y_train, cv = 3)

(-1.1255563513095836, 0.00938884627205457)

In [6]:
## fit on the train dataset and make prediction on validation data
model.fit(X_train, y_train)
model.predict(X_test)

array([3, 4, 3, ..., 3, 3, 3])

In [7]:
log_loss(y_test, model.predict_proba(X_test))

1.1340957496582915

In [8]:
model.predict_proba(X_test)

array([[0.02365559, 0.04270338, 0.53979343, 0.31985626, 0.07047925,
        0.0035121 ],
       [0.03146512, 0.03559174, 0.31712335, 0.39455462, 0.21020362,
        0.01106159],
       [0.02649378, 0.0617263 , 0.6338786 , 0.17008914, 0.10249326,
        0.00531895],
       ...,
       [0.2996721 , 0.20697536, 0.4342054 , 0.03396275, 0.02328528,
        0.00189909],
       [0.16746572, 0.28660786, 0.4599189 , 0.06094865, 0.02244411,
        0.00261475],
       [0.31262612, 0.14112748, 0.49992508, 0.02867302, 0.0165468 ,
        0.00110148]], dtype=float32)

### Making First Submission

In [9]:
## train model on all dataset
model.fit(X, y)

XGBClassifier(objective='multi:softprob', seed=42)

In [10]:
t = test.drop(['Tour_ID', 'country'], axis =1)
t = pd.get_dummies(t)
t = t.rename(columns= {'age_group_65+':'age_group_65_plus','age_group_<18':'age_group_18_less'})

In [11]:
preds = model.predict_proba(t)

In [12]:
sub_df = pd.DataFrame({'Tour_ID':test['Tour_ID'], 'High Cost': preds[:,3],'Higher Cost': preds[:,4],'Highest Cost': preds[:,5],'Low Cost': preds[:,1],'Lower Cost': preds[:,0],'Normal Cost' :preds[:,2]})
sub_df.to_csv('base_submission',index = False)

In [13]:
sub_df


,Tour_ID,High Cost,Higher Cost,Highest Cost,Low Cost,Lower Cost,Normal Cost
0,tour_idynufedne,0.150030,0.055474,0.002302,0.091114,0.037461,0.663619
1,tour_id9r3y5moe,0.206949,0.633271,0.044926,0.009177,0.012033,0.093644
2,tour_idf6itml6g,0.543361,0.255564,0.033310,0.018804,0.008628,0.140333
3,tour_id99u4znru,0.058476,0.026701,0.001667,0.266618,0.137878,0.508660
4,tour_idj4i9urbx,0.037634,0.016211,0.001198,0.162799,0.224862,0.557296
...,...,...,...,...,...,...,...
6164,tour_id2deyfjhq,0.018498,0.007191,0.000820,0.239637,0.509637,0.224218
6165,tour_idlenv2rio,0.113247,0.055356,0.000969,0.149418,0.290790,0.390220
6166,tour_id7wwqrs0p,0.375789,0.136180,0.017687,0.031698,0.036242,0.402404
6167,tour_idx80vbw5a,0.109685,0.031070,0.001428,0.167991,0.262409,0.427417


position 72